# Assignment 9 Clara Jones


In [22]:
%matplotlib inline

from pathlib import Path

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
import matplotlib.pylab as plt
from dmba import classificationSummary, gainsChart
import seaborn as sns; sns.set()


In [23]:
acc = pd.read_csv('AccidentsFull.csv')
acc.head()

,HOUR_I_R,ALCHL_I,ALIGN_I,STRATUM_R,WRK_ZONE,WKDY_I_R,INT_HWY,LGTCON_I_R,MANCOL_I_R,PED_ACC_R,...,SUR_COND,TRAF_CON_R,TRAF_WAY,VEH_INVL,WEATHER_R,INJURY_CRASH,NO_INJ_I,PRPTYDMG_CRASH,FATALITIES,MAX_SEV_IR
0,0,2,2,1,0,1,0,3,0,0,...,4,0,3,1,1,1,1,0,0,1
1,1,2,1,0,0,1,1,3,2,0,...,4,0,3,2,2,0,0,1,0,0
2,1,2,1,0,0,1,0,3,2,0,...,4,1,2,2,2,0,0,1,0,0
3,1,2,1,1,0,0,0,3,2,0,...,4,1,2,2,1,0,0,1,0,0
4,1,1,1,0,0,1,0,3,2,0,...,4,0,2,3,1,0,0,1,0,0


In [24]:
acc.columns

Index(['HOUR_I_R', 'ALCHL_I', 'ALIGN_I', 'STRATUM_R', 'WRK_ZONE', 'WKDY_I_R',
       'INT_HWY', 'LGTCON_I_R', 'MANCOL_I_R', 'PED_ACC_R', 'RELJCT_I_R',
       'REL_RWY_R', 'PROFIL_I_R', 'SPD_LIM', 'SUR_COND', 'TRAF_CON_R',
       'TRAF_WAY', 'VEH_INVL', 'WEATHER_R', 'INJURY_CRASH', 'NO_INJ_I',
       'PRPTYDMG_CRASH', 'FATALITIES', 'MAX_SEV_IR'],
      dtype='object')

In [41]:
acc['INJURY'] = np.where((acc['MAX_SEV_IR']==1)|(acc['MAX_SEV_IR']==2), 'yes', 'no')
acc.head()

,HOUR_I_R,ALCHL_I,ALIGN_I,STRATUM_R,WRK_ZONE,WKDY_I_R,INT_HWY,LGTCON_I_R,MANCOL_I_R,PED_ACC_R,...,TRAF_CON_R,TRAF_WAY,VEH_INVL,WEATHER_R,INJURY_CRASH,NO_INJ_I,PRPTYDMG_CRASH,FATALITIES,MAX_SEV_IR,INJURY
0,0,2,2,1,0,1,0,3,0,0,...,0,3,1,1,1,1,0,0,1,yes
1,1,2,1,0,0,1,1,3,2,0,...,0,3,2,2,0,0,1,0,0,no
2,1,2,1,0,0,1,0,3,2,0,...,1,2,2,2,0,0,1,0,0,no
3,1,2,1,1,0,0,0,3,2,0,...,1,2,2,1,0,0,1,0,0,no
4,1,1,1,0,0,1,0,3,2,0,...,0,2,3,1,0,0,1,0,0,no


## a)

In [54]:
BMY = acc.INJURY == 'yes'
BMN = acc.INJURY == 'no'
print('Number of accidents with injury: ', acc[BMY]['INJURY'].count())
print('Number of accidents without injury: ', acc[BMN]['INJURY'].count())

Number of accidents with injury:  21462
Number of accidents without injury:  20721


In [60]:
percent = acc[BMY]['INJURY'].count()/(acc[BMN]['INJURY'].count()+acc[BMY]['INJURY'].count())
percent.round(4)

0.5088

An injury should be predicted because there is a slightly greater than 50% chance that there will be an injury, based on the count of injuries in the dataframe.

## b)

### i)

In [5]:
acc2 = acc[['TRAF_CON_R','WEATHER_R','INJURY']]
acc2 = acc2.head(12)
acc2

,TRAF_CON_R,WEATHER_R,INJURY
0,0,1,yes
1,0,2,no
2,1,2,no
3,1,1,no
4,0,1,no
5,0,2,yes
6,0,2,no
7,0,1,yes
8,0,2,no
9,0,2,no


In [6]:
pd.pivot_table(acc2, values='INJURY' ,index=['TRAF_CON_R','WEATHER_R'], aggfunc='count')

INJURY
TRAF_CON_R WEATHER_R        
0          1               3
           2               6
1          1               1
           2               1
2          1               1

In [7]:
pd.pivot_table(acc2, values='INJURY' ,index=['WEATHER_R','TRAF_CON_R'], aggfunc='count')

INJURY
WEATHER_R TRAF_CON_R        
1         0                3
          1                1
          2                1
2         0                6
          1                1

In [8]:
pd.pivot_table(acc2, values='INJURY' ,index=['TRAF_CON_R'], aggfunc='count')

,INJURY
TRAF_CON_R,
0,9
1,2
2,1


In [9]:
pd.pivot_table(acc2, values='INJURY' ,index=['WEATHER_R'], aggfunc='count')

,INJURY
WEATHER_R,
1,5
2,7


In [10]:
pd.pivot_table(acc2, index=['INJURY','TRAF_CON_R','WEATHER_R'], aggfunc='count')

Empty DataFrame
Columns: []
Index: [(no, 0, 1), (no, 0, 2), (no, 1, 1), (no, 1, 2), (no, 2, 1), (yes, 0, 1), (yes, 0, 2)]

### ii)

In [101]:
BM = acc2.INJURY == 'yes'
acc3 = acc2[BM]
acc3

,TRAF_CON_R,WEATHER_R,INJURY
0,0,1,yes
5,0,2,yes
7,0,1,yes


In [103]:
pd.pivot_table(acc2, values='INJURY' ,index=['TRAF_CON_R','WEATHER_R'], aggfunc='count')

INJURY
TRAF_CON_R WEATHER_R        
0          1               3
           2               6
1          1               1
           2               1
2          1               1

In [102]:
pd.pivot_table(acc3, values='INJURY' ,index=['TRAF_CON_R','WEATHER_R'], aggfunc='count')

INJURY
TRAF_CON_R WEATHER_R        
0          1               2
           2               1

In [117]:
#injury=yes|weather=1,traf=0
b1 = (2/3 * 3/12)/(3/12)
print('injury=yes|weather=1,traf=0 =',b1)

#injury=yes|weather=1,traf=1
b2 = (0/3 * 3/12)/(1/12)
print('injury=yes|weather=1,traf=1 =',b2)

#injury=yes|weather=1,traf=2
b3 = (0/3 * 3/12)/(1/12)
print('injury=yes|weather=1,traf=2 =',b3)

#injury=yes|weather=2,traf=0
b4 = (1/3 * 3/12)/(6/12)
print('injury=yes|weather=2,traf=0 =',b4)

#injury=yes|weather=2,traf=1
b5 = (0/3 * 3/12)/(1/12)
print('injury=yes|weather=2,traf=1 =',b5)

#injury=yes|weather=2,traf=2
b6 = 'NaN'#(0/3 * 3/12)/(0.000000000001/12)
print('injury=yes|weather=2,traf=2 =',b6)


injury=yes|weather=1,traf=0 = 0.6666666666666666
injury=yes|weather=1,traf=1 = 0.0
injury=yes|weather=1,traf=2 = 0.0
injury=yes|weather=2,traf=0 = 0.16666666666666666
injury=yes|weather=2,traf=1 = 0.0
injury=yes|weather=2,traf=2 = NaN


### iii)

In [119]:
acc4 = acc2
acc4

,TRAF_CON_R,WEATHER_R,INJURY
0,0,1,yes
1,0,2,no
2,1,2,no
3,1,1,no
4,0,1,no
5,0,2,yes
6,0,2,no
7,0,1,yes
8,0,2,no
9,0,2,no


In [122]:
acc4['PROB'] = (0.667, 0.167, 0, 0, 0.667, 0.167, 0.167, 0.667, 0.167, 0.167, 0.167, 0)
acc4['PREDICTION'] = np.where((acc4['PROB']>.5), 'yes', 'no')
acc4

,TRAF_CON_R,WEATHER_R,INJURY,PROB,PREDICTION
0,0,1,yes,0.667,yes
1,0,2,no,0.167,no
2,1,2,no,0.000,no
3,1,1,no,0.000,no
4,0,1,no,0.667,yes
5,0,2,yes,0.167,no
6,0,2,no,0.167,no
7,0,1,yes,0.667,yes
8,0,2,no,0.167,no
9,0,2,no,0.167,no


### iv)

In [123]:
#injury=yes|WEATHER_R = 1, TRAF_CON_R = 1
prob = (3/12 * 0/3)/(1/12)
prob

0.0

### v)

In [62]:
predictors = ['TRAF_CON_R','WEATHER_R']
outcome = 'INJURY'

X = pd.get_dummies(acc2[predictors])
y = acc2['INJURY']

#run naive Bayesian
delays_nb = MultinomialNB(alpha=0.01)
delays_nb.fit(X, y)

# predict probabilities
predProb_train = delays_nb.predict_proba(X)

# predict class membership
y_pred = delays_nb.predict(X)

In [63]:
# Subset a specific set
dfacc = pd.concat([pd.DataFrame({'actual': y, 'predicted': y_pred}),
                pd.DataFrame(predProb_train, index=y.index)], axis=1)
dfacc

,actual,predicted,0,1
0,yes,no,0.703564,0.296436
1,no,no,0.652500,0.347500
2,no,no,0.993756,0.006244
3,no,no,0.995053,0.004947
4,no,no,0.703564,0.296436
5,yes,no,0.652500,0.347500
6,no,no,0.652500,0.347500
7,yes,no,0.703564,0.296436
8,no,no,0.652500,0.347500
9,no,no,0.652500,0.347500


# Part C

### i)

We can include: 'HOUR_I_R', 'ALIGN_I', 'WRK_ZONE', 'WKDY_I_R',
       'INT_HWY', 'LGTCON_I_R', 'PROFIL_I_R', 'SPD_LIM', 'SUR_COND', 'TRAF_CON_R',
       'TRAF_WAY', 'WEATHER_R'

### ii)

In [64]:
acc = acc.astype('category')

In [65]:
predictors = ['HOUR_I_R', 'ALIGN_I', 'WRK_ZONE', 'WKDY_I_R',
       'INT_HWY', 'LGTCON_I_R', 'PROFIL_I_R', 'SPD_LIM', 'SUR_COND', 'TRAF_CON_R',
       'TRAF_WAY', 'WEATHER_R']
outcome = 'INJURY'

X = pd.get_dummies(acc[predictors])
y = acc['INJURY']
classes = list(y.cat.categories)

# split into training and validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=1)

# run naive Bayes
delays_nb = MultinomialNB(alpha=0.01)
delays_nb.fit(X_train, y_train)

# predict probabilities
predProb_train = delays_nb.predict_proba(X_train)
predProb_valid = delays_nb.predict_proba(X_valid)

# predict class membership
y_valid_pred = delays_nb.predict(X_valid)
y_train_pred = delays_nb.predict(X_train)

In [66]:
# Subset a specific set
df = pd.concat([pd.DataFrame({'actual': y_valid, 'predicted': y_valid_pred}),
                pd.DataFrame(predProb_valid, index=y_valid.index)], axis=1)
df

,actual,predicted,0,1
24895,yes,yes,0.455275,0.544725
5306,no,yes,0.471793,0.528207
21954,no,no,0.546095,0.453905
37948,yes,yes,0.425262,0.574738
39072,no,yes,0.429227,0.570773
...,...,...,...,...
20008,no,yes,0.481278,0.518722
40905,no,no,0.500476,0.499524
40952,yes,yes,0.454302,0.545698
18454,no,yes,0.432414,0.567586


In [67]:
from sklearn.metrics import confusion_matrix
con_matrix = pd.DataFrame(confusion_matrix(y_train, y_train_pred),
                               index = list(y.cat.categories),
                               columns=list(y.cat.categories))
con_matrix


,no,yes
no,4712,7680
yes,4017,8900


In [124]:
classificationSummary(y_train, y_train_pred, class_names=classes) 

Confusion Matrix (Accuracy 0.5378)

       Prediction
Actual   no  yes
    no 4712 7680
   yes 4017 8900


### iii)

In [69]:
prob_con_matrix = con_matrix/con_matrix.sum()
prob_con_matrix

,no,yes
no,0.53981,0.463209
yes,0.46019,0.536791


In [75]:
classificationSummary(y_train, y_train_pred, class_names=classes) 

print()

classificationSummary(y_valid, y_valid_pred, class_names=classes) 

Confusion Matrix (Accuracy 0.5378)

       Prediction
Actual   no  yes
    no 4712 7680
   yes 4017 8900

Confusion Matrix (Accuracy 0.5371)

       Prediction
Actual   no  yes
    no 3107 5222
   yes 2589 5956


In [76]:
error = 1-.5371
print('Overall Error: ', error)

Overall Error:  0.4629


### iv)

In [83]:
TrainError = 1-.5378
ValidError = 1-.5371
Imp = ValidError - TrainError
print('Percent improvement is: ', Imp)

Percent improvement is:  0.0006999999999999229


### v)

In [150]:
BM = acc.INJURY =='yes'

In [151]:

pvtbl1 = pd.pivot_table(acc[BM], values='INJURY' ,index=['SPD_LIM'], aggfunc='count')
pvtbl1

,INJURY
SPD_LIM,
5,4
10,11
15,90
20,92
25,1960
30,1908
35,4547
40,2326
45,3347


In [152]:

pvtbl2 = pd.pivot_table(acc, values='INJURY' ,index=['SPD_LIM'], aggfunc='count')
pvtbl2

,INJURY
SPD_LIM,
5,6
10,22
15,183
20,251
25,4205
30,3715
35,8541
40,4304
45,6587


In [153]:
BM = pvtbl1.index == 5
print(pvtbl1[BM]['INJURY']/pvtbl2['INJURY'].sum())

SPD_LIM
5    0.000095
Name: INJURY, dtype: float64


The probability of an injury when speed limit is under 5 is not exactly 0, but approximately 0 (0.000095). This is likely because <5mph is so slow that it would be unlikely for an injury to occur when the car is going at such a low speed.